## 8. Make sure you have copied the workspace archive (.zip) from the aggregator node to the collaborator node.

In [1]:
!pip install paramiko scp

from paramiko import SSHClient, AutoAddPolicy
from scp import SCPClient
from pathlib import Path
from getpass import getpass

afqdn = 'nnlicv431.inn.intel.com' # 'example.domain.com'
workspace_src = '/home/itrushkin/workspace' # 'path/to/workspace'
archive_name = 'workspace.zip'
folder_name = archive_name.split('.')[0]
workspace_dst = '/home/itrushkin'
src_user = 'itrushki'


ssh_client = SSHClient()
ssh_client.set_missing_host_key_policy(AutoAddPolicy())
ssh_client.connect(afqdn, username=src_user, password=getpass(f'Enter {src_user}@{afqdn} password:'))
scp = SCPClient(ssh_client.get_transport())
scp.get(str(Path(workspace_src) / archive_name), workspace_dst)

Enter itrushki@nnlicv431.inn.intel.com password:········


# 10. Import the workspace archive.

In [2]:
%cd $workspace_dst
!rm -rf $folder_name
!fx workspace import --archive $archive_name
%cd $folder_name

/home/itrushkin
Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-04 18:39:07.002738: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-04 18:39:07.002762: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Workspace workspace.zip has been imported.
You may need to copy your PKI certificates to join the federation.

 ✔️ OK
/home/itrushkin/workspace


## 12. Create a collaborator certificate request.

In [3]:
!fx collaborator generate-cert-request -n one -d 1
scp.put('col_one_to_agg_cert_request.zip', workspace_src)

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-04 18:39:10.236013: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-04 18:39:10.236040: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Creating COLLABORATOR certificate key pair with following settings: CN=one, SAN=DNS:one
  Moving COLLABORATOR certificate to: cert/col_one
Archive col_one_to_agg_cert_request.zip with certificate signing request created
This file should be sent to the certificate authority (typically hosted by the aggregator) for signing

 ✔️ OK


## 15. Get signed certificates from the aggregator.

In [4]:
scp.get(str(Path(workspace_src) / 'agg_to_col_one_signed_cert.zip'), '.')
!fx collaborator certify --import agg_to_col_one_signed_cert.zip

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-04 18:39:26.095952: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-04 18:39:26.095978: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Certificate col_one.crt installed to PKI directory

 ✔️ OK


# Choose a device

In [5]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


# Run the collaborator process

In [6]:
!fx collaborator start -n one

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-04 18:41:01.793787: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-04 18:41:01.793813: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[18:41:02] INFO     Loading DEFAULTS for section aggregator from file plan/defaults/aggregator.yaml.                                                  ]8;id=1607096462.7709932-887857;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:87
           INFO     Loading DEFAULTS for section assigner from file plan/defaults/assigner.yaml.                                                      ]8;id=1607096462.7728684-222337;file:///home/itrushkin/.virtualenvs/fledge/l

           INFO     Settings 🡆 {'batch_size': 128, 'collaborator_count': 2, 'data_group_name': 'histology', 'data_path': '1'}                        ]8;id=1607096462.8083804-548952;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:148
           INFO     Override 🡆 {}                                                                                                                    ]8;id=1607096462.8097897-393644;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:149
[18:41:04] INFO     Histology > X_train Shape : (2000, 3, 150, 150)                                                                       ]8;id=1607096464.4890707-653442;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:136
           INFO     Histology > y_train Shape : (2000,)                                                                                

E1204 18:41:06.724409976   32035 ssl_transport_security.cc:1395] Handshake failed with fatal error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED.
EXCEPTION : <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1607096466.724526996","description":"Failed to pick subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":3948,"referenced_errors":[{"created":"@1607096466.724523580","description":"failed to connect to all addresses","file":"src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":394,"grpc_status":14}]}"
>
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /home/itrushkin/.virtualenvs/fledge/bin/fx:8 in <module>                     │
│                                                                              │
│ /home/itrushkin/.virtu